- https://www.glassdoor.com/developer/index.htm
- https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+scientes&sc.keyword=data+scientest&locT=&locId=&jobType=
- https://github.com/arapfaik/scraping-glassdoor-selenium
- https://medium.com/@jamievaron/to-anyone-who-has-lost-themselves-9c5e3049cb13
- https://git.generalassemb.ly/charles-rice/SEA-Flex-11/tree/master/08_week/selenium-webscraping
- https://devhints.io/xpath

In [1]:
# load in dataset


In [82]:
# imports
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep

In [83]:
# load in dataset
df = pd.read_csv('../data/main_df.csv')

In [84]:
df.head(2)

,disasterNumber,state,incidentType,year,month,occ_code,occ_title,tot_emp,h_mean,a_mean,employment_rate_during,employment_rate_before,employment_rate_after,employment_rate_change,wage_change
0,1190,NE,Severe Storm(s),1997.0,11.0,13002,Financial Managers,3730.0,24.50,50960,71.5,71.5,71.6,-0.1,-43864.800
1,1190,NE,Severe Storm(s),1997.0,11.0,13005,"Personnel, Training, and Labor Relations Managers",1420.0,21.41,44540,71.5,71.5,71.6,-0.1,-14593.056


In [85]:
df.columns

Index(['disasterNumber', 'state', 'incidentType', 'year', 'month', 'occ_code',
       'occ_title', 'tot_emp', 'h_mean', 'a_mean', 'employment_rate_during',
       'employment_rate_before', 'employment_rate_after',
       'employment_rate_change', 'wage_change'],
      dtype='object')

In [86]:
df.dtypes

disasterNumber              int64
state                      object
incidentType               object
year                      float64
month                     float64
occ_code                   object
occ_title                  object
tot_emp                   float64
h_mean                    float64
a_mean                     object
employment_rate_during    float64
employment_rate_before    float64
employment_rate_after     float64
employment_rate_change    float64
wage_change               float64
dtype: object

In [87]:
# create industry code categories
# Code from Patrick Kajibale 
df['Business_type']= df['occ_code'].apply(lambda x : x[:2])
#pd.set_option('display.max_row()',None)
industry = {
        '13' :'Business and Financial Operations',
        '15' :'Computer and Mathematical',
        '17' :'Architecture and Engineering',
        '19' :'Life, Physical, and Social Science',
        '21' :'Community and Social Service',
        '23' :'Legal',
        '25' :'Educational Instruction and Library',
        '27' :'Arts, Design, Entertainment, Sports, and Media',
        '29' :'Healthcare Practitioners and Technical',
        '31' :'Healthcare Support',
        '33' :'Protective Service',
        '35' :'Food Preparation and Serving Related',
        '37' :'Building and Grounds Cleaning and Maintenance',
        '39' :'Personal Care and Service',
        '41' :'Sales and Related',
        '43' :'Office and Administrative Support',
        '45' :'Farming, Fishing, and Forestry',
        '47' :'Construction and Extraction',
        '49' :'Installation, Maintenance, and Repair',
        '51' :'Production',
        '53' :'Transportation and Material Moving'}
df['Business_type'].replace(industry, inplace = True)

In [88]:
df.head(2)
## we'll keep business_type categorization in case we need it for scrape

,disasterNumber,state,incidentType,year,month,occ_code,occ_title,tot_emp,h_mean,a_mean,employment_rate_during,employment_rate_before,employment_rate_after,employment_rate_change,wage_change,Business_type
0,1190,NE,Severe Storm(s),1997.0,11.0,13002,Financial Managers,3730.0,24.50,50960,71.5,71.5,71.6,-0.1,-43864.800,Business and Financial Operations
1,1190,NE,Severe Storm(s),1997.0,11.0,13005,"Personnel, Training, and Labor Relations Managers",1420.0,21.41,44540,71.5,71.5,71.6,-0.1,-14593.056,Business and Financial Operations


### Data Scrape - Selenium

In [ ]:
# first step 
## scrape one full page, see what our results are

Notes:
- I am not experiencing a log-in prompt every time I click, I will include this code as a net in case this is dependent on mine own machine
- `options.add_argument(‘headless’)` to bypass Chrome window opening
- full search url example:
 - `https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+scientist&sc.keyword=data+scientist&locT=&locId=&jobType=`
 <br><br> What we need is: 
 - `https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=` + *JOBNAME* + `'&locT=&locId=&jobType="`
 
 #### Blank search form:
- https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&locT=&locId=&jobType=&context=Jobs&sc.keyword=&dropdown=0

Search Dictionary:
- `typedKeyword=&` >> captures job search input in search bar
- `sc.keyword=&` >> mimics above
- `locT=&`
- `locId=&`
- `jobType=`

In [89]:
# set url for test
# this url will take us to a blank search page with 'Jobs' and 'Location' 
url = 'https://www.glassdoor.com/Job/index.htm'

In [16]:
requests.get(url)

<Response [403]>

https://www.glassdoor.com/Job/jobs.htm?sc.keyword=
data+scientist&
locT=C


In [38]:
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)

In [33]:
# this opens upa. page with two search criteria, Job Title, and Area. 
# we will have to mimic clicking and entering our inputs for each category

### Inspect Elements:

#### Job Search Bar
**Xpath**<br>`//*[@id="LocationSearch"]//div[1]` <br>
**HTML** <br> `<input id="LocationSearch" class="loc" type="text" tabindex="0" value="Bellevue, WA" data-srch-type="popular" data-test="search-bar-location-input" placeholder="Location" aria-label="Location">`

#### Location Search Bar
**Xpath** <br> `//*[@id="LocationSearch"]`
**HTML** <br> `<input id="LocationSearch" class="loc" type="text" tabindex="0" value="Bellevue, WA" data-srch-type="popular" data-test="search-bar-location-input" placeholder="Location" aria-label="Location">`


## 2nd attempt, via new hyperlink
#### Job Search Bar
**Xpath**<br>`//*[@id="sc.keyword"]` <br>
**HTML** <br> `<input name="sc.keyword" id="sc.keyword" class="keyword" type="text" tabindex="0" value="" placeholder="Job Title, Keywords, or Company" data-auto-complete="true" data-ac-version="New" data-test="search-bar-keyword-input" aria-label="Keyword" autocomplete="off">`

#### Location Search Bar
**Xpath** <br> `//*[@id="sc.location"]`
**HTML** <br> `<input id="sc.location" class="loc" type="text" tabindex="0" value="Bellevue, WA" data-srch-type="popular" data-test="search-bar-location-input" placeholder="Location" aria-label="Location" autocomplete="off">`

In [ ]:
## THis run is a fail 

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')
# this is only needed if page waits too long
# driver.find_element_by_class_name("qual_x_svg_X").click()
kw = driver.find_element_by_xpath('//*[@id="KeywordSearch"]')
loc = driver.find_element_by_xpath('//*[@id="LocationSearch"]')

In [79]:

def job_track(keyword, location = None):
    # query chrome driver for selenium
    driver = webdriver.Chrome(
        executable_path="./chromedriver/macos/chromedriver")
    
    # execute driver on url
    driver.get(url)
    
    
    #locates the search field specifying 'Job Title, Keyewards, or Company'
    kw = driver.find_element_by_xpath('//*[@id="KeywordSearch"]')
    kw.clear()
    kw.send_keys(keyword)
    sleep(2)
    #locates the search field specifying 'Location'
    ## current issue is html/glassdoor default your location by mac/ip
    ## so we are manually entering our location search as a possible field
    loc = driver.find_element_by_xpath('//*[@id="LocationSearch"]')
    loc.clear()
    loc.send_keys(location)
    sleep(2)
    loc.send_keys(Keys.RETURN)
    
    # and we xpath the search button to click search
#     click = driver.find_element_by_xpath('//*[@id="HeroSearchButton"]')
#     click.click()
#     sleep(10)
#     driver.close()


In [80]:
job_track('data scientist', location = 'Virginia')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="KeywordSearch"]"}
  (Session info: chrome=81.0.4044.138)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.14.6 x86_64)


Findings: So, this works, but regardless of location input glassdoor overwrites your location with their handling.

## changing url to that of returned results (still has access to search bars)
Our idea is to test the autocompletion and bypass it by being able to locate and input our location query, adding lag delay and clicking first option from the search menu.

In [92]:
# changing url - this one works
url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+scientist&sc.keyword=data+scientist&locT=C&locId=1138213&jobType="
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
sleep(2)
driver.close()
# this is only needed if page waits too long
# driver.find_element_by_class_name("qual_x_svg_X").click()
# kw = driver.find_element_by_xpath('//*[@id="KeywordSearch"]')
# loc = driver.find_element_by_xpath('//*[@id="LocationSearch"]')

In [ ]:
# test copy/paste for testing 
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
# this is only needed if page waits too long
# driver.find_element_by_class_name("qual_x_svg_X").click()
# kw = driver.find_element_by_xpath('//*[@id="KeywordSearch"]')
# loc = driver.find_element_by_xpath('//*[@id="LocationSearch"]')

In [95]:
## working function!!!!

def job_track(keyword, location = None):
    ''' This function simply takes a job name and location, opens up glassdoor
    and searches your criteria
    
    ensure'''
    # query chrome driver for selenium
    driver = webdriver.Chrome(
        executable_path="./chromedriver/macos/chromedriver")
    
    # execute driver on url
    driver.get(url)
    
    
    #locates the search field specifying 'Job Title, Keyewards, or Company'
    kw = driver.find_element_by_xpath('/html/body/header/div[3]/div[2]/form/input[5]')
    kw.clear()
    kw.send_keys(keyword)
    sleep(1)
    kw.send_keys(u'\ue015') # down arrow
    sleep(1)
    kw.send_keys(u'\ue004') # tab to select closest match
    sleep(1)
    #locates the search field specifying 'Location'
    ## current issue is html/glassdoor default your location by mac/ip
    ## so we are manually entering our location search as a possible field
    loc = driver.find_element_by_xpath('/html/body/header/div[3]/div[2]/form/input[6]')
    loc.clear()
    loc.send_keys(location)
    sleep(1)
    loc.send_keys(u'\ue015') # down arrow
    sleep(1)
    loc.send_keys(u'\ue004') # tab to select closest match
    sleep(1)
    
    # and we xpath the search button to click search
    click = driver.find_element_by_xpath('//*[@id="HeroSearchButton"]/span')
    click.click()
#     sleep(20)
#     driver.close()



In [98]:
job_track("data scientist", 'bellevue')

In [109]:
# let's pull a job's total listings in a city
def total_listings(job, location= None):
    job_track(job, location)
    try:
        return driver.find_element_by_xpath('//*[@id="MainColSummary"]/div/div/div[2]').text
    except:
        return print("Please check your search criteria")

In [110]:
total_listings('data scientest', 'Bellevue WA')

Please check your search criteria


#### Job Search Bar
**Xpath**<br>`//*[@id="sc.keyword"]` <br>
**HTML** <br> `<input name="sc.keyword" id="sc.keyword" class="keyword" type="text" tabindex="0" value="" placeholder="Job Title, Keywords, or Company" data-auto-complete="true" data-ac-version="New" data-test="search-bar-keyword-input" aria-label="Keyword" autocomplete="off">`

#### Location Search Bar
**Xpath** <br> `//*[@id="sc.location"]`
**HTML** <br> `<input id="sc.location" class="loc" type="text" tabindex="0" value="Bellevue, WA" data-srch-type="popular" data-test="search-bar-location-input" placeholder="Location" aria-label="Location" autocomplete="off">`

In [49]:
# second step
## assess what information we can pull from a single page
## guage time frames timelines - looks like one month tops for glass door selenium



In [ ]:
# third step
## build function to scrape multiple pages

In [ ]:
# key points

## categorize jobs

## find stats on openings/hirings

## count of listings




In [ ]:
# What are we doing?

# Build a scraper/tool that compiles job listings and builds a dataframe.

## inputs: category, state, job name

## outputs: data frame
### columns: 

### Example Codes

- https://github.com/arapfaik/scraping-glassdoor-selenium
- I don't like this because it specifies location in driver. I need to fix that

In [ ]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

- From SEA-FLEX-11 review with Charlie Rice

In [ ]:
df = pd.DataFrame(columns=['name', 'location', 'price', 'cuisine','rating','reviews'])

# one big for loop!
for row in soup.find_all('div', {'class': 'rest-row-info'}):
    name = row.find('span', {'class':'rest-row-name-text'}).text
    loc = row.find('span',{'class':'rest-row-meta--location rest-row-meta-text sfx1388addContent'}).text
    price = int(row.find('i', {'class':'pricing--the-price'}).text.count('$'))
    cuisine = row.find('span', {'class':'rest-row-meta--cuisine rest-row-meta-text sfx1388addContent'}).text
    try:
        rating = row.find('div',{'class':'star-rating-score'}).attrs['aria-label'].rsplit('s')[0].strip()
    except:
        rating = 0
    try:
        reviews = row.find('a',{'class':'review-link'}).find('span').text.strip('()')
    except:
        reviews = 0
    df.loc[len(df)] = [name, loc, price, cuisine, rating, reviews]
    

df.head()